In [1]:
from transformers import AutoProcessor, Pix2StructForConditionalGeneration

# * #
model = Pix2StructForConditionalGeneration.from_pretrained("google/matcha-base")
# * #
processor = AutoProcessor.from_pretrained("google/matcha-base")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# incase u have changed helpers make sure to reload again - uncomment below two lines

# import importlib,sys
# importlib.reload(sys.modules["helpers"])

from helpers import create_metadata

# To create metadata file aas per imagefolder format
# * table_path,dest_path #
create_metadata(table_path="train/tables",dest_path="Dataset/train")

/Users/yvvsj/Desktop/Tarkas Work/Project/helpers.py:60: SyntaxWarning: invalid escape sequence '\|'
  headers = re.split('\|',lines[0])
/Users/yvvsj/Desktop/Tarkas Work/Project/helpers.py:65: SyntaxWarning: invalid escape sequence '\|'
  metrics = re.split('\|',line)


In [5]:
from datasets import load_dataset

# * data_dir #
dataset = load_dataset("imagefolder",data_dir="Dataset",split="train")

Generating train split: 75 examples [00:00, 11217.52 examples/s]


In [8]:
import torch
from torch.utils.data import Dataset, DataLoader

MAX_PATCHES = 1024
processor2 = AutoProcessor.from_pretrained("ybelkada/pix2struct-base")

class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        # changed encoding to be consistent with deplot model
        # * #
        encoding = self.processor(images=item["image"],text = "Generate Underlying data table of figure below ", return_tensors="pt", add_special_tokens=True, max_patches=MAX_PATCHES)
        
        encoding = {k:v.squeeze() for k,v in encoding.items()}
        encoding["text"] = item["text"]
        return encoding
    
def collator(batch):
  new_batch = {"flattened_patches":[], "attention_mask":[]}
  texts = [item["text"] for item in batch]
  # changed processor for text processing to remove the error, but might not be consistent
  text_inputs = processor2(images= None,text=texts, padding="max_length", return_tensors="pt", add_special_tokens=True, max_length=20,truncation=True)
  
  new_batch["labels"] = text_inputs.input_ids
  
  for item in batch:
    new_batch["flattened_patches"].append(item["flattened_patches"])
    new_batch["attention_mask"].append(item["attention_mask"])
  
  new_batch["flattened_patches"] = torch.stack(new_batch["flattened_patches"])
  new_batch["attention_mask"] = torch.stack(new_batch["attention_mask"])

  return new_batch



train_dataset = ImageCaptioningDataset(dataset, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=5,collate_fn=collator)

In [9]:
EPOCHS = 1

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

model.train()

for epoch in range(EPOCHS):
  print("Epoch:", epoch)
  for idx, batch in enumerate(train_dataloader):
    # remove below line to run for all batches
    if(idx > 1) : continue
    labels = batch.pop("labels").to(device)
    flattened_patches = batch.pop("flattened_patches").to(device)
    attention_mask = batch.pop("attention_mask").to(device)

    outputs = model(flattened_patches=flattened_patches,
                    attention_mask=attention_mask,
                    labels=labels)
    
    loss = outputs.loss

    print("Loss:", loss.item())

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

    if (epoch + 1) % 20 == 0:
        model.eval()

        predictions = model.generate(flattened_patches=flattened_patches, attention_mask=attention_mask)        
        print("Predictions:", processor.batch_decode(predictions, skip_special_tokens=True))

        model.train()

Epoch: 0


ValueError: Invalid image type. Expected either PIL.Image.Image, numpy.ndarray, torch.Tensor, tf.Tensor or jax.ndarray, but got <class 'NoneType'>.

In [10]:
total_params = sum(p.numel() for p in model.parameters())
print(total_params)

282285696
